In [1]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
np.random.seed(123)
n_samples = 1000
n_features = 100
ipw_effects = []
ra_effects = []
dom_effects = []
psm_effects = []
dr_effects = []
for iteration in tqdm(range(1000)):
    # Generate synthetic high-dimensional covariate data
    X = np.random.normal(0, 1, size=(n_samples, n_features))

    # Simulate treatment assignment influenced by the first five covariates
    logit = np.dot(X[:, :5], np.random.rand(5)) - 1  # Adjusted to simulate different probabilities
    T = np.random.binomial(1, p=1 / (1 + np.exp(-logit)))

    # Simulate the outcome where treatment has a hypothetical positive effect
    Y = 10 + np.dot(X[:, :5], np.random.rand(5)) + 5 * T + np.random.normal(0, 1, size=n_samples)


    from sklearn.linear_model import LogisticRegression, LinearRegression
    from econml.dr import DRLearner

    # Specify propensity score model
    propensity_model = LogisticRegression()
    # Specify outcome model
    outcome_model = LinearRegression()
    # Setup and fit the doubly robust estimator
    dr_learner = DRLearner(model_propensity=propensity_model, model_regression=outcome_model, model_final=LinearRegression())
    dr_learner.fit(Y, T, X=X)

    # Estimate the causal effect
    effects = dr_learner.effect(X)

    # Estimating propensity scores
    logit_ipw = LogisticRegression()
    logit_ipw.fit(X, T)
    pscores = logit_ipw.predict_proba(X)[:, 1]

    # Calculating weights
    weights = T / pscores + (1 - T) / (1 - pscores)

    # Weighted regression for outcome
    model_ipw = GradientBoostingRegressor()
    model_ipw.fit(X, Y, sample_weight=weights)
    predicted_ipw = model_ipw.predict(X)

    # Estimating the effect
    ipw_effect = np.mean(predicted_ipw[T == 1]) - np.mean(predicted_ipw[T == 0])

    # Nearest neighbor matching
    nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X[T == 0])
    _, indices = nn.kneighbors(X[T == 1])

    # Calculate average effect on matched data
    matched_controls = Y[T == 0][indices.flatten()]
    matched_treated = Y[T == 1]
    psm_effect = np.mean(matched_treated - matched_controls)

    # Direct outcome model
    model_dom = LinearRegression()
    model_dom.fit(np.hstack((X, T.reshape(-1, 1))), Y)
    predicted_dom = model_dom.predict(np.hstack((X, T.reshape(-1, 1))))

    # Estimating the effect
    dom_effect = np.mean(predicted_dom[T == 1]) - np.mean(predicted_dom[T == 0])

    dr_effects.append(np.mean(effects))
    ipw_effects.append(ipw_effect)
    dom_effects.append(dom_effect)
    psm_effects.append(psm_effect)


100%|██████████| 1000/1000 [1:18:46<00:00,  4.73s/it]


In [2]:
import numpy as np
from scipy import stats

for results in [dr_effects, ipw_effects, dom_effects, psm_effects]:
    # Calculate mean and standard deviation
    mean = np.mean(results)
    std_dev = np.std(results)

    # Calculate standard error
    std_error = stats.sem(results)

    # Calculate 95% confidence interval
    ci = stats.t.interval(0.95, len(results) - 1, loc=mean, scale=std_error)
    print(ci[0])
    print(f"Mean: {mean:.2f} ")
    print(f"95% Confidence Interval ({ci[0]:.2f}, {ci[1]:.2f})")

4.4806640348239535
Mean: 5.00 
95% Confidence Interval (4.48, 5.52)
3.2055144917062144
Mean: 3.23 
95% Confidence Interval (3.21, 3.25)
5.929658452504605
Mean: 5.95 
95% Confidence Interval (5.93, 5.97)
5.728815145871025
Mean: 5.75 
95% Confidence Interval (5.73, 5.77)
